In [1]:
import pandas as pd
import pickle

from typing import Dict

In [46]:
# df = pd.read_pickle('/data/edmitrie/alfa_boosters/net_atrs_first_net/part-00009.parquet.pckl')

# from sklearn.preprocessing import LabelEncoder

# lb_enc.fit(df[df['target'] !='nan']['target'].unique())
# lb_enc = LabelEncoder()

# lb_enc.fit(df[df['target'] !='nan']['target'].unique())

In [67]:
lb_encoder = pickle.load(open('target_enc_2_net.pickle','rb'))

In [63]:
df = pd.read_pickle('/data/edmitrie/alfa_boosters/net_atrs_first_net/part-00009.parquet.pckl')

In [64]:
# df = pd.read_pickle('/data/edmitrie/alfa_boosters/df_atr_1_net.pickle')

In [65]:
df = df[df['target'] != 'nan']

In [68]:
df['target'] = lb_encoder.transform(df['target'])

In [70]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [71]:
# df['timestamp']

In [72]:
%%time
df = df.sort_values(by='timestamp')

CPU times: user 6.35 s, sys: 582 ms, total: 6.93 s
Wall time: 6.91 s


## Формирование валидационной,тестовой,трейн выборок

In [73]:
import numpy as np

In [74]:
gr_cl = df.groupby('client_pin').size()

In [75]:
# use_clnts = pd.Series(gr_cl[gr_cl > 3].index)
use_clnts = pd.Series(gr_cl[gr_cl > 2].index)

In [76]:
train_test_df = df.merge(use_clnts, on='client_pin', how='inner')

In [77]:
train_df = train_test_df.groupby('client_pin').apply(lambda x: x.iloc[:-1]).reset_index(drop='True')
# valid_df = train_test_df.groupby('client_pin').apply(lambda x: x.iloc[:-1]).reset_index(drop='True')

In [78]:
# valid_df.sort_values(by='timestamp', inplace=True)
train_df.sort_values(by='timestamp', inplace=True)

In [79]:
test_df = train_test_df

In [80]:
test_df.sort_values(by='timestamp', inplace=True)

In [104]:
a = pd.read_csv('alfabattle2_abattle_train_target.csv')

In [105]:
a.head()

,session_id,client_pin,timestamp,multi_class_target
0,0000029e72e5fcde6a9f29c3a3ed198f,7cf9221322a0e2fdefb1b998b8f2ab29,2020-06-15 14:01:12,main_screen
1,00063dffa47b0fe5556b2b9e8beddb6a,5f16c0ab27a806fd08db3122921adf3a,2020-03-21 12:59:34,invest
2,0007857f36d268ec46fcb7305538a1c7,ec868fc2b388293cf10e18ee9518d72f,2020-01-24 18:18:55,statement
3,000a7f25fc3609cdfda54c5f059aab00,91f55a33d7502c1a1fa5da7ff2f7b648,2020-03-15 19:50:23,main_screen
4,000b746d6616669663feaa1474ac97f1,3ef1020bda95ce7836d2680fa553ecb7,2020-02-20 07:56:58,main_screen


## Формирование сэмплов под нейронку

In [82]:
def truncate(x, num_last_transactions=750):
    return x.values.transpose()[:, -num_last_transactions:].tolist()


def transform_transactions_to_sequences(transactions_frame: pd.DataFrame,atrs,
                                        num_last_transactions=750) -> pd.DataFrame:
    return transactions_frame \
        .groupby(['client_pin'])[atrs] \
        .apply(lambda x: truncate(x, num_last_transactions=num_last_transactions)) \
        .reset_index().rename(columns={0: 'sequences'})


In [83]:
def pad_sequence(array, max_len) -> np.array:
    add_zeros = int(max_len - len(array[0]))
    return np.array([list(x) + [0] * add_zeros for x in array])



def create_padded_buckets(frame_of_sequences: pd.DataFrame, bucket_info: Dict[int, int],
                          save_to_file_path=None, has_target=True):
    frame_of_sequences['bucket_idx'] = frame_of_sequences.sequence_length.map(bucket_info)
        
    
#     print(frame_of_sequences.sequence_length.max(), frame_of_sequences.sequence_length.min())
        
    padded_seq = []
    targets = []
    lengths = []
    client_pin = []
    
#     print(frame_of_sequences.session_id.unique().shape)
#     print(frame_of_sequences.bucket_idx.isnull().sum())

    
    from tqdm import tqdm

    for size, bucket in frame_of_sequences.groupby('bucket_idx'):
        padded_sequences = bucket.sequences.apply(lambda x: pad_sequence(x, size)).values
        padded_sequences = np.array([np.array(x) for x in padded_sequences])
        padded_seq.append(padded_sequences)
        
        if has_target:
            targets.append(bucket.target.values)
            
#         print(bucket.sequence_length.shape)
            
        lengths.append(bucket.sequence_length.values)
        client_pin.append(bucket.client_pin.values)        
    

    frame_of_sequences.drop(columns=['bucket_idx'], inplace=True)
    
    
    if has_target:
        dict_result = {
            'padded_sequences':np.array(padded_seq),
            'targets':np.array(targets),
            'lengths': np.array(lengths),
            'client': np.array(client_pin),
        }
    else:
         dict_result = {
            'padded_sequences':np.array(padded_seq),
            'lengths': np.array(lengths),
            'client': np.array(client_pin),
        }

    if save_to_file_path:
        with open(save_to_file_path, 'wb') as f:
            pickle.dump(dict_result, f)
    return dict_result


def create_buckets_from_transactions(df, file_name, submit=False):   
    import tqdm
    import os
    
    train_df = df
    
    if not submit:
        target = df.groupby('client_pin')[['target','session_id', 'timestamp']].last().reset_index()

    #     return target

        train_df = df.merge(target[['session_id']], on='session_id', how='left', indicator=True)
        train_df = train_df[train_df['_merge'] == 'left_only']
        
        train_df.drop(columns='_merge', inplace=True)
        
    seq = transform_transactions_to_sequences(train_df, atrs)
    seq['sequence_length'] = seq.sequences.apply(lambda x: len(x[1]))
    
    if not submit:                
        seq = seq.merge(target[['client_pin', 'target']], how='left', on='client_pin')
    
    mapping_seq_len_to_padded_len = pickle.load(open('buckets_info.pkl', 'rb'))
    
    if not submit:
        processed_fragment =  create_padded_buckets(seq, mapping_seq_len_to_padded_len, has_target=True)    
    else:
        processed_fragment =  create_padded_buckets(seq, mapping_seq_len_to_padded_len, has_target=False)    
    res = pd.DataFrame(processed_fragment)
    res.to_pickle(file_name)
    
    return res

In [84]:
def get_data_for_net(file_name):
    df = pd.read_pickle(file_name)
    
    df = df[df['target'] != 'nan']
    
    import pickle
    
    lb_encoder = pickle.load(open('target_enc_2_net.pickle','rb'))
    
    df['target'] = lb_encoder.transform( df['target'])
    
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.sort_values(by='timestamp', inplace=True)
    
    atrs = [col for col in df.columns if col not in ['client_pin', 'session_id', 'target', 'timestamp']]
    
    last_name = file_name.split('/')[-1]
    
    print('приступаю к сабмит файлу')
    
    create_buckets_from_transactions(df, f'/data/edmitrie/alfa_boosters/net_atrs_2_net/{last_name}_submit', submit=True)
    
    print('сабмит сохранил')
    
    gr_cl = df.groupby('client_pin').size()
    use_clnts = pd.Series(gr_cl[gr_cl > 2].index)
    
    train_test_df = df.merge(use_clnts, on='client_pin', how='inner')
    
    train_df = train_test_df.groupby('client_pin').apply(lambda x: x.iloc[:-1]).reset_index(drop='True')
    test_df = df
    
    train_df.sort_values(by='timestamp', inplace=True)
    test_df.sort_values(by='timestamp', inplace=True)

    create_buckets_from_transactions(train_df, f'/data/edmitrie/alfa_boosters/net_atrs_2_net/{last_name}_train')
    
    print('сохранил трейн')
    
    create_buckets_from_transactions(test_df, f'/data/edmitrie/alfa_boosters/net_atrs_2_net/{last_name}_test')  
    print('сохранил тест')
    
    

In [85]:
# import os

# DIR = '/data/edmitrie/alfa_boosters/net_atrs_first_net'

# for file in os.listdir():
    

In [86]:
%%time
# get_data_for_net('/data/edmitrie/alfa_boosters/net_atrs_first_net/part-00008.parquet.pckl')

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.39 µs


In [87]:
%%time
test_df_net = create_buckets_from_transactions(test_df, 'chunks_data/processed_test_2_net.pkl')
train_df_net = create_buckets_from_transactions(train_df, 'chunks_data/processed_train_2_net.pkl')
# valid_df_net = create_buckets_from_transactions(valid_df, 'chunks_data/processed_valid_2_net.pkl')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:44: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:45: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:46: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local

In [88]:
# test_df_net = pd.read_pickle('chunks_data/processed_test_2_net.pkl')

## Моделирование

In [89]:
import torch
import pandas as pd

In [90]:
test_df_net = pd.read_pickle('chunks_data/processed_test_2_net.pkl')
train_df_net = pd.read_pickle('chunks_data/processed_train_2_net.pkl')
# valid_df_net = pd.read_pickle('chunks_data/processed_valid_2_net.pkl')

In [91]:
def batches_generator(data,batch_size=32, shuffle=False, is_infinite=False,
                      verbose=False, device=None, output_format='torch', is_train=True):
    while True:
#         if shuffle:
#             np.random.shuffle(list_of_paths)

#         for path in list_of_paths:
#             if verbose:
#                 print(f'reading {path}')

        padded_sequences, targets, lengths = data['padded_sequences'], data['targets'], data['lengths']

        indices = np.arange(len(targets))

        if shuffle:
            np.random.shuffle(indices)

            padded_sequences = padded_sequences[indices]
            targets = targets[indices]
            lengths = lengths[indices]

        for idx in range(len(padded_sequences)):

            bucket = padded_sequences[idx]

            if is_train:
                target = targets[idx]
                length = lengths[idx]

            for jdx in range(0, len(bucket), batch_size):
                batch_sequences = bucket[jdx: jdx + batch_size]
#                 print(target.shape)


                if is_train:
                    batch_targets = target[jdx: jdx + batch_size]
                    batch_length = length[jdx: jdx + batch_size]

                batch_sequences = torch.FloatTensor(batch_sequences).to(device)

                if is_train:
                    yield dict(transactions_features=batch_sequences,
                               label=torch.LongTensor(batch_targets).to(device),
                               lengths=torch.LongTensor(batch_length).to(device)
                              )
                else:
                    yield dict(transactions_features=batch_sequences,
                               product=torch.LongTensor(batch_products).to(device),
                               lengths=torch.LongTensor(data['lengths']).to(device)
                               )
        if not is_infinite:
            break

In [92]:
from sklearn.metrics import f1_score

In [93]:
device='cuda'

In [94]:
def train_epoch(model, optimizer, dataset_train, valid_dataset,  batch_size=64, shuffle=True,
                print_loss_every_n_batches=100, device=None):
    train_generator = batches_generator(dataset_train, batch_size=batch_size, shuffle=shuffle,
                                        device=device, is_train=True, output_format='torch')
    
    loss_function = nn.CrossEntropyLoss()
    
    num_batches = 1
    running_loss = 0.0

    model.train()

    for i, batch in enumerate(train_generator):
        
        output = model(batch['transactions_features'], batch['lengths'])
#         print(output.shape)
#         print(batch['label'].shape)
        batch_loss = loss_function(output, batch['label'])
        
        if i % 100 == 0:
#             val_generator = batches_generator(dataset_valid, atrs, batch_size=batch_size, shuffle=False,
#                                       device=device, is_train=True, output_format='torch')
            model_os = []
            targets_l = []
                        
            for (seqs, targets, lengths) in zip(valid_dataset['padded_sequences'], valid_dataset['targets'], valid_dataset['lengths']):
                batch_sequences = torch.FloatTensor(seqs).to(device)
                lengths = torch.LongTensor(lengths).to(device)
    
                logits = model(batch_sequences, lengths)
    
                model_os.append(logits.argmax(axis=-1).detach().cpu().numpy().flatten())
                targets_l.append(targets)
#                 print(np.unique(logits.argmax(axis=-1).detach().cpu().numpy().flatten()))

            print(f1_score(np.concatenate(model_os), np.concatenate(targets_l), average='macro'))
            
#             for batch_t in val_generator:
                
#                 _, output = model(batch_t['transactions_features'], batch_t['lengths'])
#                 metrics.append(f1_score(batch_t['label'].detach().cpu().numpy().flatten(), output.argmax(axis=-1).detach().cpu().numpy().flatten(), average='macro'))
                
#             print(np.mean(metrics))
#             print(metrics)

        batch_loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        running_loss += batch_loss

        if num_batches % print_loss_every_n_batches == 0:
            print(f'Training loss after {num_batches} batches: {running_loss / num_batches}', end='\r')
        
        num_batches += 1
    
    print(f'Training loss after epoch: {running_loss / num_batches}', end='\r')

In [95]:
from torch import nn

class TransactionsRnn(nn.Module):
    def __init__(self,  rnn_units=300, top_classifier_units=32, classes=11):
        super(TransactionsRnn, self).__init__()
                                
        self._gru = nn.GRU(input_size=128, num_layers=1,
                             hidden_size=rnn_units, batch_first=True, bidirectional=False)
        
        self._hidden_size = rnn_units
                
        self._top_classifier = nn.Linear(in_features=rnn_units, 
                                         out_features=top_classifier_units)
        
        self._intermediate_activation = nn.ReLU()
        
        self._head = nn.Linear(in_features=top_classifier_units, out_features=classes)
        
    
    def forward(self, input_t, lengths): 
        input_t = input_t.permute(0, 2, 1)
#         print(input_t.shape)
        
        packed_inputs = torch.nn.utils.rnn.pack_padded_sequence(input_t, lengths, batch_first=True, enforce_sorted=False)
            
        _, final_hiddens = self._gru(packed_inputs)
                
        final_hiddens = final_hiddens.view(input_t.size(0),-1)
                                           
        classification_hidden = self._top_classifier(final_hiddens)
        activation = self._intermediate_activation(classification_hidden)
        
        logits = self._head(activation)
        
        return logits

In [96]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [97]:
# device = 'cpu'

In [98]:
model = TransactionsRnn()

In [99]:
model = model.to(device)

In [100]:
optimizer = torch.optim.Adam(lr=1e-3, params=model.parameters())

In [101]:
num_epochs = 200
train_batch_size = 128
val_batch_szie = 256

In [102]:
import numpy as np

In [103]:
for epoch in range(num_epochs):
    print(f'Starting epoch {epoch+1}')
    
    train_epoch(model, optimizer, train_df_net, test_df_net,  batch_size=train_batch_size, 
                shuffle=True, print_loss_every_n_batches=500, device=device)
    
#     val_roc_auc = eval_model(model, dataset_val, batch_size=val_batch_szie, device=device)
#     es(val_roc_auc, model)

#     if es.early_stop:
#         print('Early stopping reached. Stop training...')
#         break
#     torch.save(model.state_dict(), os.path.join(path_to_checkpoints, f'epoch_{epoch+1}_val_{val_roc_auc:.3f}.pt'))
    
#     train_roc_auc = eval_model(model, dataset_train, batch_size=val_batch_szie, device=device)
#     print(f'Epoch {epoch+1} completed. Train roc-auc: {train_roc_auc}, Val roc-auc: {val_roc_auc}')

Starting epoch 1
0.03052272418217149
0.23391162965607165
Starting epoch 2ter epoch: 1.481927514076233
0.266917537032637
0.27965360138248346
Starting epoch 3ter epoch: 1.2918602228164673
0.2829478642854447
0.30109129297267917
Starting epoch 4ter epoch: 1.2184216976165771
0.30170555708838076
0.3171363832866937
Starting epoch 5ter epoch: 1.1636524200439453
0.29427722573154996
0.31837939531937265
Starting epoch 6ter epoch: 1.1147105693817139
0.28998297194848
0.3204360582718024
Starting epoch 7ter epoch: 1.0595520734786987
0.3256090741811337
0.3354545160271575
Starting epoch 8ter epoch: 1.0106464624404907
0.3243756554030078
0.3337535216814425
Starting epoch 9ter epoch: 0.9533373713493347
0.33353563075616127
0.32673296621276954
Starting epoch 10er epoch: 0.9087492823600769
0.34568485285135975
0.3133815329727134
Starting epoch 11er epoch: 0.8563689589500427
0.3621976665229255
0.30455785774012617
Starting epoch 12er epoch: 0.8023719787597656
0.3568100524766362
0.29765976865984495
Starting epoc

In [94]:
device = 'cpu'

In [95]:
model.to(device)

TransactionsRnn(
  (_gru): GRU(128, 200, batch_first=True)
  (_top_classifier): Linear(in_features=200, out_features=32, bias=True)
  (_intermediate_activation): ReLU()
  (_head): Linear(in_features=32, out_features=11, bias=True)
)

In [128]:
model_os = []
targets_l = []

for sequnces, targets, lengths in zip(test_df_net['padded_sequences'], test_df_net['targets'], test_df_net['lengths']):
    batch_sequences = torch.FloatTensor(sequnces).to(device)
    lengths = torch.LongTensor(lengths).to(device)
    
    logits = model(batch_sequences, lengths)
    
    model_os.append(logits.argmax(axis=-1).detach().cpu().numpy().flatten())
    targets_l.append(targets)

In [139]:
outputs = np.concatenate(model_os)

In [140]:
targets = np.concatenate(targets_l)

In [141]:
pd.DataFrame(classification_report(targets, outputs, output_dict=True)).T

/home/edmitrie/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,precision,recall,f1-score,support
0,0.333333,0.038462,0.068966,260.000000
1,0.438849,0.204698,0.279176,298.000000
2,0.407407,0.028796,0.053790,382.000000
3,0.557258,0.679449,0.612317,1017.000000
4,0.000000,0.000000,0.000000,54.000000
5,0.588089,0.809034,0.681091,3210.000000
6,0.540541,0.058140,0.104987,344.000000
8,0.421053,0.455696,0.437690,316.000000
9,0.340741,0.224939,0.270987,409.000000
10,0.455310,0.372633,0.409844,1162.000000
